In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Paths and stuff
import os
import sys

sys.path.append('/home/shreyas/pySICOPOLIS/src')
from pySICOPOLIS import *

In [2]:
vel_data_40kms = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/vel_data_40kms.nc")

In [3]:
sicopolis_dir = '/home/shreyas/update_to_develop_sicopolis/sicopolis_tuneCS'
simulation = 'grl40_bm5_paleo17a_nudged_CT4_BH0_SVC_m21ka_pkp'
dict_sico_out_folder_prefixes = {"nodiff": "N",
                                 "tlm": "FORWARD",
                                 "adj": "ADJOINT",
                                 "tlm_action": "FORWARDHESSACTION",
                                 "adj_action": "ADJHESSACTION"}
dict_ad_exec_cmds_suffixes = {"nodiff": "nodiff",
                              "tlm": "forward",
                              "adj": "adjoint",
                              "tlm_action": "forwardhessaction",
                              "adj_action": "adjointhessaction"}
dict_ad_log_file_suffixes = {"nodiff": "nodiff",
                             "tlm": "tlm",
                             "adj": "adj",
                             "tlm_action": "tlm_hessaction",
                             "adj_action": "adj_hessaction"}
dict_ad_nc_suffixes = {"nodiff": "nodiff",
                       "tlm": "tlm",
                       "adj": "adj",
                       "tlm_action": "tlm_hessaction",
                       "adj_action": "adj_hessaction"}

KCMAX = 80
exp_sigma_level = dataCleaner.exp_sigma_level(zeta = np.arange(0,1+1./KCMAX,1./KCMAX),
                                              exponent = 2.0)
xModel40       = np.arange(-72.,97.,4.0)*10
yModel40       = np.arange(-345.,-56.,4.0)*10
time_ad = np.arange(12, dtype=float)
IMAX = xModel40.shape[0]-1
JMAX = yModel40.shape[0]-1

log_c_slide_init = np.log10(13.5)*np.ones((JMAX+1, IMAX+1))

dict_og_state_fields_vals = {"xx_c_slide_init": log_c_slide_init}

dict_state_fields_num_dims = {"xx_c_slide_init": "2D"}

dict_state_coords = {"time_ad": time_ad,
               "zeta_c": exp_sigma_level,
               "y": yModel40,
               "x": xModel40}

dict_state_attrs_type = {"xx_c_slide_init": "nodiff"}

dict_state_fields_or_scalars = {"xx_c_slide_init": "field"}

dict_masks_observables = {"vs": None}

In [4]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables, 
                                None, True, "0002.nc")

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "adj")

fc = DA.eval_cost()

ds_x = DA.create_ad_tlm_action_input_nc(bool_randomize = True)
ds_x_tlm_only = DA.subset_of_ds(ds_x, "type", "tlm")
    
ds_Ax = DA.eval_tlm_action()
ds_noise_cov_inv_Ax = DA.eval_noise_cov_inv_action(ds_Ax)
ds_H_misfit_x = DA.eval_misfit_hessian_action()

DA.l2_inner_product([ds_x_tlm_only, ds_H_misfit_x], ["tlm", "adj"]), DA.l2_inner_product([ds_Ax, ds_noise_cov_inv_Ax], ["tlmhessaction", "adjhessaction"])

(3579634.9553332157, 3579634.9553016443)

In [5]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables, 
                                None, True, "0002.nc")

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "adj")

fc = DA.eval_cost()

dict_tlm_action_only_fields_vals = {}
for var in ds_x_tlm_only:
    if DA.dict_tlm_action_fields_or_scalars[var] == "scalar":
        dict_tlm_action_only_fields_vals[var] = ds_x_tlm_only[var].data[0].copy()
    else:
        dict_tlm_action_only_fields_vals[var] = ds_x_tlm_only[var].data.copy()
ds_x = DA.create_ad_tlm_action_input_nc(dict_tlm_action_only_fields_vals)
    
ds_Ax = DA.eval_tlm_action()
ds_noise_cov_inv_Ax = DA.eval_noise_cov_inv_action(ds_Ax)
ds_H_misfit_x = DA.eval_misfit_hessian_action()

DA.l2_inner_product([ds_x_tlm_only, ds_H_misfit_x], ["tlm", "adj"]), DA.l2_inner_product([ds_Ax, ds_noise_cov_inv_Ax], ["tlmhessaction", "adjhessaction"])

(3579634.9553332157, 3579634.9553016443)

In [6]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables, 
                                None, True, "0002.nc")

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "adj")

DA.gradient_descent(3, init_alpha = 1.0, min_alpha_tol = 1.e-10)

-------------------------------------
iter 0, fc = 3874738.08372731
-------------------------------------
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Step size alpha = 7.62939453125e-06
-------------------------------------
iter 1, fc = 3185208.1177812684
-------------------------------------
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed the simulation.
Too big step size probably crashed t

<xarray.Dataset> Size: 26kB
Dimensions:          (y: 73, x: 43)
Coordinates:
  * x                (x) float64 344B -7.2e+05 -6.8e+05 ... 9.2e+05 9.6e+05
  * y                (y) float64 584B -3.45e+06 -3.41e+06 ... -6.1e+05 -5.7e+05
Data variables:
    xx_c_slide_init  (y, x) float64 25kB 1.13 1.13 1.13 1.13 ... 1.13 1.13 1.13
Attributes:
    history:  2024-12-31 13:45:47 -06:00 - Data produced

In [7]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables, 
                                None, True, "0002.nc")

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "adj")

DA.inexact_gn_hessian_cg(MAX_ITERS=3, init_alpha_gd = 1.e-6)

-------------------------------------
Initial fc = 3874738.08372731
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
CG iter 4
CG iter 5
conjugate_gradient: Convergence.
Angle between p and g in degrees:  113.7614212252116
Step size alpha = 0.25
-------------------------------------
Outer iter 1, fc = 3748113.1411274867
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
conjugate_gradient: Convergence.
Angle between p and g in degrees:  163.50697510389546
Step size alpha = 1.0
-------------------------------------
Outer iter 2, fc = 3246799.6579710036
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
CG iter 4
conjugate_gradient: Convergence.
Angle between p and g in degrees:  102.58480953787883
Minimum tolerable step size alpha reached.
Step size alpha = 5.820766091346741e-11
Step size alpha 5.820766091346741e-11 is too small for any real improvement with Inexact GN-Hessian CG, switching to gradient descent for this step.
Step size

<xarray.Dataset> Size: 26kB
Dimensions:          (y: 73, x: 43)
Coordinates:
  * x                (x) float64 344B -7.2e+05 -6.8e+05 ... 9.2e+05 9.6e+05
  * y                (y) float64 584B -3.45e+06 -3.41e+06 ... -6.1e+05 -5.7e+05
Data variables:
    xx_c_slide_init  (y, x) float64 25kB ...
Attributes:
    history:  2024-12-31 13:48:14 -06:00 - Data produced

In [8]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables, 
                                None, True, "0002.nc")

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "adj")

DA.inexact_gn_hessian_cg(MAX_ITERS=3, init_alpha_gd = 1.e-6, cg_tolerance_type = "linear")

-------------------------------------
Initial fc = 3874738.08372731
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
CG iter 4
CG iter 5
conjugate_gradient: Convergence.
Angle between p and g in degrees:  113.7614212252116
Step size alpha = 0.25
-------------------------------------
Outer iter 1, fc = 3748113.1411274867
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
conjugate_gradient: Convergence.
Angle between p and g in degrees:  163.50697510389546
Step size alpha = 1.0
-------------------------------------
Outer iter 2, fc = 3246799.6579710036
-------------------------------------
CG iter 1
CG iter 2
CG iter 3
CG iter 4
conjugate_gradient: Convergence.
Angle between p and g in degrees:  102.58480953787883
Minimum tolerable step size alpha reached.
Step size alpha = 5.820766091346741e-11
Step size alpha 5.820766091346741e-11 is too small for any real improvement with Inexact GN-Hessian CG, switching to gradient descent for this step.
Step size

<xarray.Dataset> Size: 26kB
Dimensions:          (y: 73, x: 43)
Coordinates:
  * x                (x) float64 344B -7.2e+05 -6.8e+05 ... 9.2e+05 9.6e+05
  * y                (y) float64 584B -3.45e+06 -3.41e+06 ... -6.1e+05 -5.7e+05
Data variables:
    xx_c_slide_init  (y, x) float64 25kB ...
Attributes:
    history:  2024-12-31 13:49:55 -06:00 - Data produced